In [2]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment2/'
FOLDERNAME = '/content/drive/MyDrive/Stanford Research/honor thesis for Ephysics/ephysics project/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
# %cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
# !bash get_datasets.sh
# %cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# !pip install pymatgen
!pip install torchviz

from torchviz import make_dot
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

import sklearn
import csv
import pandas as pd
# import pymatgen as mg
import random
import os
from bisect import bisect_left   
from sklearn import preprocessing
from sklearn.utils import Bunch
from sklearn import svm
# from mp_api.client import MPRester

#from sklearn import datasets, svm
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import time


import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split, KFold

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, confusion_matrix


style.use("ggplot")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#shuffles dataset
def format_dataset(X, y, size = -1):
    random.seed(a=5)
    length = np.size(y)
    indices = np.arange(length)
    np.random.shuffle(indices)
    new_X = []
    new_y = []
    for x in indices:
        new_X.append(X[x])
        new_y.append(y[x])
    if size == -1:
        return Bunch(data=new_X, target=new_y)
    return Bunch(data=new_X[0:size], target=new_y[0:size])

#pos, neg are lists
def get_labels(pos, neg):
    y = []
    for i in range(len(pos)):
        y.append(1)
    for i in range(len(neg)):
        y.append(0)
    return np.array(y)


def calculate_classification_rates(predictions, labels):
    """
    Calculates the classification rates based on the predictions and labels.

    Args:
    - predictions: NumPy array of predicted values (shape: [n_samples])
    - labels: List of true labels (length: n_samples)

    Returns:
    - true_positive_rate: Float, true positive rate (TPR) or sensitivity
    - false_positive_rate: Float, false positive rate (FPR)
    - true_negative_rate: Float, true negative rate (TNR) or specificity
    - false_negative_rate: Float, false negative rate (FNR)
    """

    # Convert labels to NumPy array for easier manipulation
    labels = np.array(labels)

    # Calculate the number of true positive, false positive, true negative, and false negative
    true_positive = np.sum((predictions == 1) & (labels == 1))
    false_positive = np.sum((predictions == 1) & (labels == 0))
    true_negative = np.sum((predictions == 0) & (labels == 0))
    false_negative = np.sum((predictions == 0) & (labels == 1))

    # Calculate the rates
    true_positive_rate = true_positive / (true_positive + false_negative)
    false_positive_rate = false_positive / (false_positive + true_negative)
    true_negative_rate = true_negative / (true_negative + false_positive)
    false_negative_rate = false_negative / (false_negative + true_positive)

    return true_positive_rate, false_positive_rate, true_negative_rate, false_negative_rate

In [5]:
# Define MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(112, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        self.act = nn.ELU()

    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.fc3(x)
        return x
      

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(112, 64, kernel_size=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(64, 2)  # Assuming binary classification

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(x)
        x = self.fc(x)
        return x


#define transformer model
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(TransformerClassifier, self).__init__()
        self.transformer = nn.TransformerEncoder(nn.TransformerEncoderLayer(input_dim, nhead=4), num_layers=4)
        self.fc = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(0.2)
        self.activation = nn.ELU()
        self.output_layer = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        x = self.transformer(x)
        # x = x.mean(dim=1)  # Average pooling across time steps
        x = self.fc(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [ ]:
X_pos = np.load(FOLDERNAME + '/partial_perfect_in_icsd.npy')
X_neg = np.load(FOLDERNAME + '/not_in_icsd_proofread.npy')
X_neg = X_neg[:10899, :]

#concatenate the features together and scale
X = np.concatenate((X_pos, X_neg), axis = 0)
std_scale=preprocessing.StandardScaler().fit(X)
X = std_scale.transform(X)

#get output label
y = []

y=get_labels(X_pos,X_neg)

In [ ]:
mlp = MLP()

In [ ]:
X_tensor = torch.tensor(X, dtype=torch.float32)

In [ ]:
y_forward = mlp(X_tensor)

In [ ]:
make_dot(y_forward, params=dict(mlp.named_parameters())).render("MLP_network", format="png")

'MLP_network.png'

In [ ]:
X_pos = np.load(FOLDERNAME + '/partial_perfect_in_icsd.npy')
X_neg = np.load(FOLDERNAME + '/not_in_icsd_proofread.npy')
X_neg = X_neg[:10899, :]

#concatenate the features together and scale
X = np.concatenate((X_pos, X_neg), axis = 0)
std_scale=preprocessing.StandardScaler().fit(X)
X = std_scale.transform(X)

#get output label
y = []

y=get_labels(X_pos,X_neg)

X_tensor = torch.tensor(X, dtype=torch.float32)

In [ ]:
input = X_tensor.unsqueeze(1).permute(0,2,1)

In [ ]:
cnn = CNN()

In [ ]:
cnn_y_forward = cnn(input)

In [ ]:
make_dot(cnn_y_forward, params=dict(cnn.named_parameters())).render("CNN_network", format="png")

'CNN_network.png'

In [8]:
X = np.load(FOLDERNAME + '/partial_perfect_in_icsd.npy')
# X_neg = np.load(FOLDERNAME + '/not_in_icsd_proofread.npy')
# X_neg = X_neg[:10899, :]

#concatenate the features together and scale
# X = np.concatenate((X_pos, X_neg), axis = 0)
std_scale=preprocessing.StandardScaler().fit(X)
X = std_scale.transform(X)

#get output label
y = []

# y=get_labels(X)

X_tensor = torch.tensor(X, dtype=torch.float32)

In [9]:
trans = TransformerClassifier(input_dim=112, hidden_dim=256, num_classes=2)

In [10]:
trans_output = trans(X_tensor)

In [12]:
make_dot(trans_output, params=dict(trans.named_parameters())).render("transformer_network", format="png")

'transformer_network.png'